## refill_imaging_batch_table
It became clear that imaging batches were going to be necessary, just like clearing batches. In the notebook [fill_imaging_batch_table.ipynb](fill_imaging_batch_table.ipynb), I filled these initially. It later became clear that these tables needed to be modified and a minor database migration needed to take place. This was easier to do with python than manually in sql pro. 

In this notebook, I populate a copy of the ImagingBatch() table to have imaging request numbers, since that was the missing column from before

In [1]:
import pickle, os
import os.path, sys
from datetime import datetime
import pandas as pd
import numpy as np
import datajoint as dj
dj.config["enable_python_native_blobs"] = True # So I can store python dictionaries in blob columns

## Connect to the db

In [2]:
dj.config['database.host'] = 'datajoint00.pni.princeton.edu'
dj.config['database.user'] = os.environ.get('DJ_DB_USER')
dj.config['database.password'] = os.environ.get('DJ_DB_PASS')
dj.conn()

Connecting ahoag@datajoint00.pni.princeton.edu:3306


DataJoint connection (connected) ahoag@datajoint00.pni.princeton.edu:3306

In [3]:
# set up object for light sheet schema
db_lightsheet = dj.create_virtual_module('ahoag_lightsheet_demo','ahoag_lightsheet_demo')
# db_lightsheet = dj.create_virtual_module('u19lightserv_lightsheet','u19lightserv_lightsheet')

In [ ]:
# Here are the existing contents
imaging_batch_contents = db_lightsheet.Request.ImagingBatch()
imaging_batch_contents

In [ ]:
# All we need to do is ingest copies of these with a new column into the table ImagingBatchCopy()
insert_dict_list = imaging_batch_contents.fetch(as_dict=True)
for d in insert_dict_list:
    d['imaging_request_number'] = 1

In [ ]:
db_lightsheet.Request.ImagingBatchCopy().insert(insert_dict_list)

## Fill ClearingBatchSample() and ImagingBatchSample() tables
After making these tables and removing the imaging and clearing batch information from the sample table, I need to make inserts for these tables from the pre-existing requests in the database

In [4]:
db_lightsheet.Request().ClearingBatchSample()

username user in the lab,request_name,sample_name,clearing_protocol,antibody1,antibody2,clearing_batch_number
lightserv-test,test_new_imaging_request,test_new_imaging_request-001,iDISCO abbreviated clearing,,,1
lightserv-test,test_new_imaging_request,test_new_imaging_request-002,iDISCO abbreviated clearing,,,1


In [5]:
db_lightsheet.Request().ClearingBatch()

username user in the lab,request_name,clearing_protocol,antibody1,antibody2,clearing_batch_number,antibody1_lot,antibody2_lot,clearing_progress,number_in_batch,perfusion_date,expected_handoff_date,clearer user in the lab,notes_for_clearer,link_to_clearing_spreadsheet
afalkner,"MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6",iDISCO+_immuno,SySy Rb antiFos 1:1000,Donkey antiRabbit 488 1:500,1,None,None,complete,8,None,None,None,,https://docs.google.com/spreadsheets/d/1A83HVyy1bEhctqArwt4EiT637M8wBxTFodobbt1jrXI/edit#gid=0
ahoag,test,iDISCO abbreviated clearing,,,1,None,None,incomplete,1,None,None,ahoag,,None
ahoag,test123,iDISCO abbreviated clearing,,,1,None,None,complete,1,None,2020-01-01,ahoag,,None
ahoag,test_3p6x,iDISCO abbreviated clearing,,,1,None,None,complete,1,None,2020-01-01,ahoag,,None
ahoag,test_idiscoplus,iDISCO+_immuno,antibody 1 test,antibody 2 test,1,None,None,in progress,1,None,None,ahoag,,None
ahoag,test_large_smartspim,iDISCO abbreviated clearing,,,1,None,None,complete,1,None,None,ahoag,,None
ahoag,test_many_samples,iDISCO abbreviated clearing,,,1,None,None,in progress,10,2019-02-04,2020-01-01,ahoag,,None
ahoag,test_many_samples2,iDISCO abbreviated clearing,,,1,None,None,complete,4,None,None,ahoag,here are notes for clearer,None
ahoag,test_many_samples3,iDISCO abbreviated clearing,,,1,None,None,complete,3,None,None,ahoag,,None
ahoag,test_many_samples4,iDISCO abbreviated clearing,,,1,None,None,complete,2,None,None,ahoag,test notes,None


In [9]:
request_contents = db_lightsheet.Request().fetch(as_dict=True)
sample_contents = db_lightsheet.Request.Sample()
imaging_request_contents = db_lightsheet.Request.ImagingRequest()
channel_contents = db_lightsheet.Request.ImagingChannel()

In [10]:
clearing_batch_sample_insert_list = []
imaging_batch_sample_insert_list = []
for request_dict in request_contents:
    imaging_dict = {}
    username = request_dict['username']
    request_name = request_dict['request_name']
    print(f"Username: {username}")
    print(f"Request Name: {request_name}")
    print()
    date_submitted = request_dict['date_submitted']
    time_submitted = request_dict['time_submitted']
    clearing_batch_master_dict = {
        'username':username,
        'request_name':request_name,
        'imaging_request_date_submitted':date_submitted,
        'imaging_request_time_submitted':time_submitted,
        'imaging_progress':'incomplete'
    }
    imaging_batch_master_dict = {
        'username':username,
        'request_name':request_name,
        'imaging_request_date_submitted':date_submitted,
        'imaging_request_time_submitted':time_submitted,
        'imaging_progress':'incomplete'
    }
    # loop through samples and figure out imaging batches
    sample_contents_this_request = (sample_contents & \
        {'username':username,'request_name':request_name}).fetch(as_dict=True)
    
    for sample_dict in sample_contents_this_request:
        sample_name = sample_dict['sample_name']
        
        imaging_dict[sample_name] = {}
        channel_contents_this_sample = channel_contents & \
            {'username':username,'request_name':request_name,'sample_name':sample_name}
        for channel_dict in channel_contents_this_sample:
            image_resolution = channel_dict['image_resolution']
            channel_name = channel_dict['channel_name']
            if image_resolution in imaging_dict[sample_name].keys():
                imaging_dict[sample_name][image_resolution].append(channel_name)
            else:
                imaging_dict[sample_name][image_resolution] = [channel_name]
#     print(imaging_dict)
    # Count up numbers in each batch
    used_dicts_dict = {} # {batch_number:imaging_dict}
    batch_counter_dict = {} # {1:{'number_in_batch':1,'imaging_dict':{},}}
    batch_number = 0
    for sample_name in imaging_dict.keys():
        if imaging_dict[sample_name] in used_dicts_dict.values():
#             print(f"sample {sample_name} belongs to an existing imaging batch")
            # Figure out the imaging batch this sample belongs to
            this_batch_number = [key for key in used_dicts_dict if used_dicts_dict[key] == imaging_dict[sample_name]]

            batch_counter_dict[batch_number]['number_in_imaging_batch'] +=1
            continue
        else:
            batch_number +=1
#             print(f"sample {sample_name} is first in batch number {batch_number}")
            # make a new entry
            batch_counter_dict[batch_number]={'number_in_imaging_batch':1,
                'imaging_dict':imaging_dict[sample_name]}
            used_dicts_dict[batch_number] = imaging_dict[sample_name]
            
    # Loop through numbers_in_sample_dict and make batches
    for batch_number in batch_counter_dict.keys():
        imaging_batch_insert_dict = imaging_batch_master_dict.copy()
        imaging_batch_insert_dict['imaging_batch_number'] = batch_number
        number_in_batch = batch_counter_dict[batch_number]['number_in_imaging_batch']
        imaging_batch_insert_dict['number_in_imaging_batch'] = number_in_batch
        imaging_dict_this_batch = batch_counter_dict[batch_number]['imaging_dict']
        imaging_batch_insert_dict['imaging_dict'] = imaging_dict_this_batch
        imaging_batch_insert_list.append(imaging_batch_insert_dict)
print(imaging_batch_insert_list)
# db_lightsheet.Request.ImagingBatch.insert(imaging_batch_insert_list,skip_duplicates=True)
# imaging_dict

Username: afalkner
Request Name: MFNP2,_MFNP3,_MMNP4,_MMNP5,_MMNP6,_FMNP4,_FMNP5,_FMNP6

Username: ahoag
Request Name: test

Username: ahoag
Request Name: test123

Username: ahoag
Request Name: test_3p6x

Username: ahoag
Request Name: test_idiscoplus

Username: ahoag
Request Name: test_large_smartspim

Username: ahoag
Request Name: test_many_samples

Username: ahoag
Request Name: test_many_samples2

Username: ahoag
Request Name: test_many_samples3

Username: ahoag
Request Name: test_many_samples4

Username: ahoag
Request Name: test_many_samples5

Username: ahoag
Request Name: test_rawdata_policy

Username: ahoag
Request Name: test_udisco

Username: ahoag
Request Name: test_udisco_rat

Username: apv2
Request Name: 20190313_IBL_DiI_1

Username: apv2
Request Name: ibl_witten_04

Username: demi
Request Name: test

Username: demi
Request Name: test2

Username: demi
Request Name: test3

Username: ejdennis
Request Name: 10-13_brains,_names_TBD

Username: ejdennis
Request Name: 201905_atlas00x

KeyboardInterrupt: 